# Solar Internship Data Science

## Team: SIIL-TEAM-0002
## Problem Statement: DS-003
## Title: Predictive Maintenance for Manufacturing Equipment
### **Members:**
1. Shashank Jangde
2. Vidit Singh
3. Sushant Pandey
4. Yash Mishra

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# importing necessary librabries
import numpy as np
import pandas as pd
import copy
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from keras import models, layers
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

2024-11-12 21:39:41.774807: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-12 21:39:41.783120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731427781.792733   10227 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731427781.795521   10227 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 21:39:41.805388: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Defining some reused functions

In [5]:
def process_data():
    """
    this function will return 3 list of pandas dataframe which would contain training, testing and RUL dataset names
    """
    dir = "./CMAPSSData/"
    train_data = ['train_FD001.txt', 'train_FD002.txt', 'train_FD003.txt', 'train_FD004.txt']
    test_data = ['test_FD001.txt', 'test_FD002.txt', 'test_FD003.txt', 'test_FD004.txt']
    rul_data = ['RUL_FD001.txt', 'RUL_FD002.txt', 'RUL_FD003.txt', 'RUL_FD004.txt']
    columns = ["Unit Number", "Cycles(Time)"] + [f"operation_setting{i}" for i in range(1, 4)] + [f"sensor{i}" for i in range(1, 22)]
    
    trainDatasets = []
    testDatasets = []
    expectedRulDatasets = []


    for i in range(4):
        # Import Files
        tempTrain = pd.read_csv( dir + train_data[i], sep = " ", header = None)
        tempTest = pd.read_csv( dir + test_data[i], sep = " ", header = None)
        tempRul = pd.read_csv( dir + rul_data[i], sep = " ", header = None)

        # Cleaning Files, we remove 26, 27 and 1 as they have null values
        tempTrain.drop(inplace = True, columns = [26, 27])
        tempTest.drop(inplace = True, columns = [26, 27])
        tempRul.drop(inplace = True, columns = [1])

        # Adding Columns Names
        tempTrain.columns = tempTest.columns = columns
        tempRul.columns = ["Expected RUL"]

        # Appending to Lists
        trainDatasets.append(tempTrain)
        testDatasets.append(tempTest)
        expectedRulDatasets.append(tempRul)
    
    return trainDatasets, testDatasets, expectedRulDatasets

In [6]:
def checkBasicStructure(trainDatasets, testDatasets, expectedRulDatasets):
    print("\nDatasets Example\n")
    for i in range(4):
        print("For FD00" + str(i+1) )
        print("\nTrain Dataset")
        display(trainDatasets[i].head())
    
        print("\nTest Dataset")
        display(testDatasets[i].head())
    
        print("\nExpected RUL")
        display(expectedRulDatasets[i].head())
        print("\n")

In [7]:
def checkForNA(trainDatasets, testDatasets, expectedRulDatasets):
    print("\nDatasets Example\n")
    for i in range(4):
        print("For FD00" + str(i+1))
        print("\nTrain Dataset")
        display(trainDatasets[i].isna().sum())
    
        print("\nTest Dataset")
        display(testDatasets[i].isna().sum())
    
        print("\nExpected RUL")
        display(expectedRulDatasets[i].isna().sum())
        print("\n")

In [8]:
def findRul(Datasets):

    rulDatasets = []
    
    for i in range(4):
        # Finding Maximum RUL
        temp = copy.deepcopy(Datasets[i].iloc[:, 0:2])
        tempRul = temp.groupby(Datasets[i].columns[0])[Datasets[i].columns[1]].max()

        # Create Max RUL Dataframe
        tempRul = pd.DataFrame(tempRul)
        tempRul.columns = ["Max"]
        tempRul = pd.merge(temp, tempRul, left_on = Datasets[i].columns[0], right_on = Datasets[i].columns[0])
        tempRul["RUL"] = tempRul["Max"] - tempRul[Datasets[i].columns[1]];
        tempRul.drop(inplace = True, columns = ["Max", Datasets[i].columns[1]])
        
        # Appending to List
        rulDatasets.append(tempRul)

    return rulDatasets

In [9]:
def checkRulBasicStructure(rulDatasets):
    print("\nRUL Datasets Example\n")
    for i in range(4):
        print("For FD00" + str(i+1) )
        print("\nExpected RUL")
        display(rulDatasets[i].head())
        print("\n")

In [10]:
def processTrainingData(dataset, rulDataset = None, window = 1, shift = 1):
    partitions = int(np.floor((len(dataset) - window) / shift)) + 1
    features = dataset.shape[1]

    processedData = np.repeat(np.nan, repeats = partitions * window * features).reshape(partitions, window, features)

    if rulDataset is None:
        for i in range(partitions):
            processedData[i, :, :] = dataset[(0 + shift * i): (0 + shift * i + window), :]
            
        return processedData

    processedRul = np.repeat(np.nan, repeats = partitions)
    for i in range(partitions):
        processedData[i, :, :] = dataset[(0 + shift * i): (0 + shift * i + window), :]
        processedRul[i] = rulDataset[(shift * i + (window - 1))]

    return processedData, processedRul

In [11]:
def processTestingData(dataset, window, shift, testWindow = 1):
    maxPartitions = int(np.floor((len(dataset) - window) / shift)) + 1

    if maxPartitions < testWindow:
        newLen = (maxPartitions - 1) * shift + window

        processedData = processTrainingData(dataset[-newLen:, :] , window = window, shift = shift)

        return processedData, maxPartitions

    newLen = (testWindow - 1) * shift + window

    processedData = processTrainingData(dataset[-newLen:, :] , window = window, shift = shift)

    return processedData, testWindow

In [12]:
def createModel(shape, name = "Combined"):
    model = Sequential(name = name)
    model.add(LSTM(128, return_sequences = True, input_shape = (shape[1], shape[2])))
    model.add(LSTM(64 ,return_sequences = True))
    model.add(LSTM(32))
    model.add(Dense(1))
    model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001))

    return model

## Loading and Checking Dataset

In [14]:
trainDatasets, testDatasets, expectedRulDatasets = process_data()

### Check Basic Structure

In [16]:
checkBasicStructure(trainDatasets, testDatasets, expectedRulDatasets)


Datasets Example

For FD001

Train Dataset


,Unit Number,Cycles(Time),operation_setting1,operation_setting2,operation_setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044



Test Dataset


,Unit Number,Cycles(Time),operation_setting1,operation_setting2,operation_setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130



Expected RUL


...

,Unit Number,Cycles(Time),operation_setting1,operation_setting2,operation_setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754



Test Dataset


,Unit Number,Cycles(Time),operation_setting1,operation_setting2,operation_setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,1,20.0072,0.7000,100.0,491.19,606.67,1481.04,1227.81,9.35,...,313.03,2387.78,8048.98,9.2229,0.02,362,2324,100.00,24.31,14.7007
1,1,2,24.9984,0.6200,60.0,462.54,536.22,1256.17,1031.48,7.05,...,163.61,2028.09,7863.46,10.8632,0.02,306,1915,84.93,14.36,8.5748
2,1,3,42.0000,0.8420,100.0,445.00,549.23,1340.13,1105.88,3.91,...,129.98,2387.95,8071.13,9.3960,0.02,328,2212,100.00,10.39,6.4365
3,1,4,42.0035,0.8402,100.0,445.00,549.19,1339.70,1107.26,3.91,...,129.48,2387.90,8078.89,9.3594,0.02,328,2212,100.00,10.56,6.2367
4,1,5,35.0079,0.8400,100.0,449.44,555.10,1353.04,1117.80,5.48,...,181.82,2387.87,8057.83,9.3030,0.02,333,2223,100.00,14.85,8.9326



Expected RUL


,Expected RUL
0,22
1,39
2,107
3,75
4,149


### Check For NAs

In [18]:
checkForNA(trainDatasets, testDatasets, expectedRulDatasets)


Datasets Example

For FD001

Train Dataset


Unit Number           0
Cycles(Time)          0
operation_setting1    0
operation_setting2    0
operation_setting3    0
sensor1               0
sensor2               0
sensor3               0
sensor4               0
sensor5               0
sensor6               0
sensor7               0
sensor8               0
sensor9               0
sensor10              0
sensor11              0
sensor12              0
sensor13              0
sensor14              0
sensor15              0
sensor16              0
sensor17              0
sensor18              0
sensor19              0
sensor20              0
sensor21              0
dtype: int64


Test Dataset


Unit Number           0
Cycles(Time)          0
operation_setting1    0
operation_setting2    0
operation_setting3    0
sensor1               0
sensor2               0
sensor3               0
sensor4               0
sensor5               0
sensor6               0
sensor7               0
sensor8               0
sensor9               0
sensor10              0
sensor11              0
sensor12              0
sensor13              0
sensor14              0
sensor15              0
sensor16              0
sensor17              0
sensor18              0
sensor19              0
sensor20              0
sensor21              0
dtype: int64


Expected RUL


...

Unit Number           0
Cycles(Time)          0
operation_setting1    0
operation_setting2    0
operation_setting3    0
sensor1               0
sensor2               0
sensor3               0
sensor4               0
sensor5               0
sensor6               0
sensor7               0
sensor8               0
sensor9               0
sensor10              0
sensor11              0
sensor12              0
sensor13              0
sensor14              0
sensor15              0
sensor16              0
sensor17              0
sensor18              0
sensor19              0
sensor20              0
sensor21              0
dtype: int64


Test Dataset


Unit Number           0
Cycles(Time)          0
operation_setting1    0
operation_setting2    0
operation_setting3    0
sensor1               0
sensor2               0
sensor3               0
sensor4               0
sensor5               0
sensor6               0
sensor7               0
sensor8               0
sensor9               0
sensor10              0
sensor11              0
sensor12              0
sensor13              0
sensor14              0
sensor15              0
sensor16              0
sensor17              0
sensor18              0
sensor19              0
sensor20              0
sensor21              0
dtype: int64


Expected RUL


Expected RUL    0
dtype: int64

## Preprocessing Datasets

In [20]:
trainDatasetsCopy = copy.deepcopy(trainDatasets)
testDatasetsCopy = copy.deepcopy(testDatasets)
expectedRulDatasetsCopy = copy.deepcopy(expectedRulDatasets)

### Converting to Standardize Distribution

In [22]:
scaler = []

for i in range(4):
    sc = StandardScaler()
    scaler.append(sc)

In [23]:
for i in range(4):
    trainDatasetsCopy[i].iloc[:, 2:] = scaler[i].fit_transform(trainDatasetsCopy[i].iloc[:, 2:])
    testDatasetsCopy[i].iloc[:, 2: ] = scaler[i].transform(testDatasetsCopy[i].iloc[:, 2:])

for i in range(4):
    pickle.dump(scaler[i], open("pcascaler/" + f"scaler{i+1}.pkl", 'wb'))

/tmp/ipykernel_10227/3663732451.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.78170979 -0.78170979 -2.07309423 ...  3.09244354  1.15536688
  1.8010591 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  trainDatasetsCopy[i].iloc[:, 2:] = scaler[i].fit_transform(trainDatasetsCopy[i].iloc[:, 2:])
/tmp/ipykernel_10227/3663732451.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.78170979 -0.13601757 -0.13601757 ...  1.15536688  1.15536688
  1.8010591 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  testDatasetsCopy[i].iloc[:, 2: ] = scaler[i].transform(testDatasetsCopy[i].iloc[:, 2:])
/tmp/ipykernel_10227/3663732451.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.51557895 

In [24]:
checkBasicStructure(trainDatasetsCopy, testDatasetsCopy, expectedRulDatasetsCopy)


Datasets Example

For FD001

Train Dataset


,Unit Number,Cycles(Time),operation_setting1,operation_setting2,operation_setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,1,-0.315980,-1.372953,0.0,0.0,-1.721725,-0.134255,-0.925936,-1.776357e-15,...,0.334262,-1.058890,-0.269071,-0.603816,-1.387779e-17,-0.781710,0,0.0,1.348493,1.194427
1,1,2,0.872722,-1.031720,0.0,0.0,-1.061780,0.211528,-0.643726,-1.776357e-15,...,1.174899,-0.363646,-0.642845,-0.275852,-1.387779e-17,-0.781710,0,0.0,1.016528,1.236922
2,1,3,-1.961874,1.015677,0.0,0.0,-0.661813,-0.413166,-0.525953,-1.776357e-15,...,1.364721,-0.919841,-0.551629,-0.649144,-1.387779e-17,-2.073094,0,0.0,0.739891,0.503423
3,1,4,0.324090,-0.008022,0.0,0.0,-0.661813,-1.261314,-0.784831,-1.776357e-15,...,1.961302,-0.224597,-0.520176,-1.971665,-1.387779e-17,-0.781710,0,0.0,0.352598,0.777792
4,1,5,-0.864611,-0.690488,0.0,0.0,-0.621816,-1.251528,-0.301518,-1.776357e-15,...,1.052871,-0.780793,-0.521748,-0.339845,-1.387779e-17,-0.136018,0,0.0,0.463253,1.059552



Test Dataset


,Unit Number,Cycles(Time),operation_setting1,operation_setting2,operation_setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,1,1.055599,1.015677,0.0,0.0,0.678077,-0.853550,-1.191480,-1.776357e-15,...,0.415614,-0.919841,-0.954235,-0.985107,-1.387779e-17,-0.781710,0,0.0,0.241943,0.774097
1,1,2,-1.230366,-1.031720,0.0,0.0,-1.941707,-0.338137,-1.501467,-1.776357e-15,...,1.012195,-0.502695,-0.216648,-1.649034,-1.387779e-17,-0.136018,0,0.0,1.127183,0.941305
2,1,3,0.141213,0.333211,0.0,0.0,-0.441831,-0.584426,-0.843717,-1.776357e-15,...,0.754581,-0.919841,-0.715712,0.052112,-1.387779e-17,-0.136018,0,0.0,1.459148,1.172256
3,1,4,1.924266,-0.008022,0.0,0.0,-0.481827,-1.044384,-0.279297,-1.776357e-15,...,-0.045381,-0.641744,-0.568929,-1.345067,-1.387779e-17,-1.427402,0,0.0,1.016528,0.775945
4,1,5,0.644125,-0.008022,0.0,0.0,-0.341839,-0.543650,-0.779276,-1.776357e-15,...,0.998637,-0.919841,-0.745069,-1.041101,-1.387779e-17,-2.073094,0,0.0,0.961200,1.138999



Expected RUL


...

,Unit Number,Cycles(Time),operation_setting1,operation_setting2,operation_setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,1,1.218156,0.864668,0.418783,-1.054690,-0.796416,-0.701412,-0.745729,-1.137677,...,-0.989007,0.417814,0.081921,0.063831,-0.694278,-0.638665,-0.114203,0.418783,-1.030999,-1.031756
1,1,2,-0.270478,0.414718,0.418783,0.692508,0.713666,0.562449,0.298212,0.363906,...,0.331131,0.415786,-0.253086,-0.125677,-0.694278,0.476120,0.655708,0.418783,0.352814,0.358264
2,1,3,1.218082,0.867565,0.418783,-1.054690,-0.815965,-0.704332,-0.711202,-1.137677,...,-0.990162,0.417658,-0.013912,0.153387,-0.694278,-0.674626,-0.114203,0.418783,-1.045089,-1.022649
3,1,4,1.217824,0.864668,0.418783,-1.054690,-0.822660,-0.722040,-0.702990,-1.137677,...,-0.988862,0.418048,0.096162,0.068362,-0.694278,-0.710586,-0.114203,0.418783,-1.039051,-1.023269
4,1,5,0.068094,0.158844,-2.387873,-0.391216,-1.160079,-1.532181,-1.410627,-0.270955,...,-0.741097,-2.389666,-2.358027,2.066982,-0.694278,-1.537685,-2.155843,-2.387873,-0.687814,-0.644612



Test Dataset


,Unit Number,Cycles(Time),operation_setting1,operation_setting2,operation_setting3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,1,-0.270126,0.414074,0.418783,0.692508,0.729733,0.594757,0.217006,0.363906,...,0.334308,0.416176,-0.219818,-0.083564,-0.694278,0.512081,0.655708,0.418783,0.346775,0.365895
1,1,2,0.067560,0.156591,-2.387873,-0.391216,-1.156865,-1.523327,-1.428310,-0.270955,...,-0.744708,-2.389588,-2.385341,2.102432,-0.694278,-1.501724,-2.155843,-2.387873,-0.654602,-0.661484
2,1,3,1.217824,0.871105,0.418783,-1.054690,-0.808467,-0.732495,-0.804811,-1.137677,...,-0.987563,0.417502,0.038732,0.147123,-0.694278,-0.710586,-0.114203,0.418783,-1.054147,-1.020100
3,1,4,1.218061,0.865312,0.418783,-1.054690,-0.809538,-0.736545,-0.793246,-1.137677,...,-0.991173,0.417112,0.129313,0.098347,-0.694278,-0.710586,-0.114203,0.418783,-1.037038,-1.053608
4,1,5,0.744765,0.864668,0.418783,-0.886741,-0.651273,-0.610894,-0.704917,-0.704316,...,-0.613207,0.416878,-0.116515,0.023184,-0.694278,-0.530782,-0.038587,0.418783,-0.605288,-0.601477



Expected RUL


,Expected RUL
0,22
1,39
2,107
3,75
4,149


### Doing Principal Component Analysis

In [26]:
pca = PCA(n_components = 10)

In [44]:
newColumns = ['PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10']

for i in range(4):
    # Finding Principal Components
    temp1 = pca.fit_transform(trainDatasetsCopy[i].iloc[:, 2:])
    
    temp2 = pca.transform(testDatasetsCopy[i].iloc[:, 2:])

    # Converting to Dataframes
    temp1 = pd.DataFrame(temp1, columns = newColumns)
    temp2 = pd.DataFrame(temp2, columns = newColumns)

    # Dropping Excess Data
    trainDatasetsCopy[i].drop(inplace = True, columns = trainDatasetsCopy[i].columns[2:])
    testDatasetsCopy[i].drop(inplace = True, columns = testDatasetsCopy[i].columns[2:])

    # Merging New Data
    trainDatasetsCopy[i] = pd.merge(trainDatasetsCopy[i], temp1, left_index=True, right_index=True)
    testDatasetsCopy[i] = pd.merge(testDatasetsCopy[i], temp2, left_index=True, right_index=True)
pickle.dump(pca, open("pcascaler/" + f"pca.pkl", 'wb'))

In [ ]:
checkBasicStructure(trainDatasetsCopy, testDatasetsCopy, expectedRulDatasetsCopy)

## Finding Training Results

In [ ]:
trainRulDatasets = findRul(trainDatasets)

In [ ]:
checkRulBasicStructure(trainRulDatasets)

## Data Processing

### Constant Variables

In [ ]:
defaultWindow = 10
defaultShift = 1
defaultTestWindow = 10

processedTrainDatasets = []
processedTestDatasets = []
processedTrainRulDatasets = []
rulDatasets = []
testWindowList = []

predictedRulDatasets = []
rmseList = []
r2_scoreList = []

### Normalizing Data

In [ ]:
scaler_model = []

for i in range(4):
    sc = StandardScaler()
    scaler_model.append(sc)

pickle.dump(scaler[i], open("pcascaler/" + f"complete_scaler.pkl", 'wb'))

In [ ]:
for i in range(4):
    trainDatasetsCopy[i].iloc[:, 1:] = scaler_model[i].fit_transform(trainDatasetsCopy[i].iloc[:, 1:])
    testDatasetsCopy[i].iloc[:, 1:] = scaler_model[i].transform(testDatasetsCopy[i].iloc[:, 1:])

In [ ]:
checkBasicStructure(trainDatasetsCopy, testDatasetsCopy, expectedRulDatasetsCopy)

In [ ]:
trainDatasetsUnique = [len(x['Unit Number'].unique()) for x in trainDatasetsCopy]
testDatasetsUnique = [len(x['Unit Number'].unique()) for x in testDatasetsCopy]

In [ ]:
display(trainDatasetsUnique)
display(testDatasetsUnique)

### Heap Map

In [ ]:
for i in range(4):
    print("For FD00" + str(i+1) + "\n")
    sns.heatmap(trainDatasetsCopy[i].corr(), annot=True, cmap="Reds", linewidths=0.2)
    fig = plt.gcf()
    fig.set_size_inches(12, 12)
    plt.show()
    print("\n")

### Processing Training Data

In [ ]:
for i in range(4):
    temp1 = []
    temp2 = []
    for j in range(1, trainDatasetsUnique[i]+1):
        tempTrainDataset = trainDatasetsCopy[i][trainDatasetsCopy[i]['Unit Number'] == j].drop(columns = ['Unit Number']).values

        window_size = min(len(tempTrainDataset), defaultWindow)

        tempTrainRulDataset = trainRulDatasets[i][trainRulDatasets[i]['Unit Number'] == j].drop(columns = ['Unit Number']).values

        tempTrainDataset, tempTrainRulDataset = processTrainingData(tempTrainDataset, tempTrainRulDataset, window = window_size, shift = defaultShift)

        temp1.append(tempTrainDataset)
        temp2.append(tempTrainRulDataset)
        
    processedTrainDatasets.append(np.concatenate(temp1))
    processedTrainRulDatasets.append(np.concatenate(temp2))

### Processing Test Data

In [ ]:
for i in range(4):
    temp1 = []
    temp2 = []
    for j in range(1, testDatasetsUnique[i]+1):
        tempTestDataset = testDatasetsCopy[i][testDatasetsCopy[i]['Unit Number'] == j].drop(columns = ['Unit Number']).values

        window_size = min(len(tempTestDataset), defaultWindow)

        tempTestDataset, testWindow = processTestingData(tempTestDataset, window = window_size, shift = defaultShift, testWindow = defaultTestWindow)

        temp1.append(tempTestDataset)
        temp2.append(testWindow)
        
    processedTestDatasets.append(np.concatenate(temp1))
    testWindowList.append(temp2)
    rulDatasets.append(expectedRulDatasetsCopy[i]["Expected RUL"].values)

### Showing Processed Data Shapes

In [ ]:
for i in range(4):
    print("For FD00" + str(i+1) + "\n")
    print("Train Dataset Shape: ", processedTrainDatasets[i].shape)
    print("Train RUL Dataset Shape: ", processedTrainRulDatasets[i].shape)
    print("Test Dataset Shape: ", processedTestDatasets[i].shape)
    print("Expected RUL Shape: ", rulDatasets[i].shape)
    print("\n")

## LSTM Individual Model Training

In [ ]:
trainedModel = []
histories = []

In [ ]:
for i in range(4):
    print("For FD00" + str(i+1) + "\n")
    name = "FD00" + str(i+1)
    model = createModel(processedTrainDatasets[i].shape, name = name)
    display(model.summary())
    history = model.fit(processedTrainDatasets[i], processedTrainRulDatasets[i], epochs = 100, validation_split = 0.2, batch_size = 128, verbose = 1)
    print("\n")
    trainedModel.append(model)
    histories.append(history)

## Individual Model Testing

### Predicting Values

In [ ]:
for i in range(4):
    predictedRul = trainedModel[i].predict(processedTestDatasets[i]).reshape(-1)
    predictedRul = np.split(predictedRul, np.cumsum(testWindowList[i])[:-1])
    predictedRul = [np.average(ruls, weights = np.repeat(1/testWindow, testWindow)) for ruls, testWindow in zip(predictedRul, testWindowList[i])]

    predictedRulDatasets.append(predictedRul)

### Finding Root Mean Square Error

In [ ]:
for i in range(4):
    rmse = np.sqrt(mse(rulDatasets[i], predictedRulDatasets[i]))
    print("For FD00" + str(i+1))
    print("Root Mean Square Error:", rmse)
    print("\n")
    rmseList.append(rmse)

### Finding R2 Score

In [ ]:
for i in range(4):
    r2_sc = r2_score(rulDatasets[i], predictedRulDatasets[i])
    print("For FD00" + str(i+1))
    print("R2 Score:", r2_sc)
    print("\n")
    r2_scoreList.append(r2_sc)

#### Expected RUL vs Predicted RUL

In [ ]:
for i in range(4):
    print("For FD00" + str(i+1) + "\n")
    plt.figure(figsize = (15, 6))
    plt.plot(rulDatasets[i], label = "Expected RUL", color = "Red")
    plt.plot(predictedRulDatasets[i], label = "Predicted RUL", color = "Green")
    plt.legend(loc = 'upper left')
    plt.show()
    print("\n")

In [ ]:
for i in range(4):
    print("For FD00" + str(i+1) + "\n")
    plt.figure(figsize = (15, 6))
    plt.plot(histories[i].history['loss'])
    plt.plot(histories[i].history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc = 'upper left')
    plt.show()
    print("\n")

## LSTM Complete Model Training

In [ ]:
completeTrainDatasets = np.concatenate(processedTrainDatasets)
completeTrainRulDatasets = np.concatenate(processedTrainRulDatasets)
completeTestDatasets = np.concatenate(processedTestDatasets)
completeRulDatasets = np.concatenate(rulDatasets)
completeTestWindowList = np.concatenate(testWindowList)

### Shapes

In [ ]:
print("Train Dataset Shape: ", completeTrainDatasets.shape)
print("Train RUL Dataset Shape: ", completeTrainRulDatasets.shape)
print("Test Dataset Shape: ", completeTestDatasets.shape)
print("Expected RUL Shape: ", completeRulDatasets.shape)

In [ ]:
completeModel = createModel(completeTrainDatasets.shape)
completeHistory = completeModel.fit(completeTrainDatasets, completeTrainRulDatasets, epochs = 100, validation_split = 0.2, batch_size = 128, verbose = 1)

### Predicting Values

In [ ]:
completePredictedRul = completeModel.predict(completeTestDatasets).reshape(-1)
completePredictedRul = np.split(completePredictedRul, np.cumsum(completeTestWindowList)[:-1])
completePredictedRul = [np.average(ruls, weights = np.repeat(1/testWindow, testWindow)) for ruls, testWindow in zip(completePredictedRul, completeTestWindowList)]

### Finding Root Mean Square Error

In [ ]:
rmse = np.sqrt(mse(completeRulDatasets, completePredictedRul))
print("Root Mean Square Error:", rmse)

### Finding R2 Score

In [ ]:
r2_sc = r2_score(completeRulDatasets, completePredictedRul)
print("R2 Score:", r2_sc)

### Visualization

#### Expected RUL vs Predicted RUL

In [ ]:
plt.figure(figsize = (15, 6))
plt.plot(completeRulDatasets, label = "Expected RUL", color = "Red")
plt.plot(completePredictedRul, label = "Predicted RUL", color = "Green")
plt.legend(loc = 'upper left')
plt.show()

#### Training History

In [ ]:
plt.figure(figsize = (15, 6))
plt.plot(completeHistory.history['loss'])
plt.plot(completeHistory.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc = 'upper left')
plt.show()

## Saving Models

In [ ]:
modelDumpFolder = 'models/'

modelDumpFiles = ['FD001.pkl', 'FD002.pkl', 'FD003.pkl', 'FD004.pkl', 'complete.pkl']

In [ ]:
for i in range(4):
    pickle.dump(trainedModel[i], open(modelDumpFolder + modelDumpFiles[i], 'wb'))

In [ ]:
pickle.dump(completeModel, open(modelDumpFolder + modelDumpFiles[i], 'wb'))